## ZheHUANG AI 22208901

# inforet 2023 4

- experimental session
- finding multi-word terminology by statistics
- matching terminology from a list
- training a NER for terminology

In [1]:
import re, regex, timeit, gzip, random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
tqdm.pandas()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from nltk.tokenize import MWETokenizer
from nltk.util import Trie

import spacy
print('spacy version: {}'.format(spacy.__version__))

from collections import defaultdict

spacy version: 3.5.1


In [2]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 638 µs (started: 2023-04-06 21:43:44 +00:00)


In [3]:
path = '/kaggle/input/ir-week-4-dataset'

time: 561 µs (started: 2023-04-06 21:43:44 +00:00)


### are frequent n-grams terms?
#### political domain:

In [4]:
df = pd.read_csv(path + "/un-general-debates-blueprint.csv")
display(df.shape)
df.sample(11)

(7507, 7)

,session,year,country,country_name,speaker,position,text
221,27,1972,ESP,Spain,Mr. Lopez-Bravo,NaN,"Mr. President, before addressing this twenty- ..."
6975,68,2013,DZA,Algeria,Ramtane Lamamra,Minister for Foreign Affairs,I would \nlike first of all to warmly congratu...
3096,47,1992,EGY,Egypt,Mr. MOUSSA,NaN,"At the outset, I \nwish to convey the Egyptian..."
5726,61,2006,RUS,Russian Federation,Mr. Sergey V. Lavrov,Minister for Foreign Affairs,"Traditionally, the General Assembly session \n..."
153,26,1971,PHL,Philippines,Mr. ROMULO,NaN,"51.\t Mr. President, your unanimous election ..."
4473,55,2000,BFA,Burkina Faso,Youssouf Ouedraogo,Minister for Foreign Affairs,"Exactly one week ago, the Millennium\nSummit w..."
549,29,1974,TUR,Turkey,Mr. Gune,NaN,"Mr. President, first I should like to extend m..."
469,29,1974,EGY,Egypt,Mr. Fahmy,NaN,"Mr. President, it is a source of pride and gra..."
2792,45,1990,GUY,Guyana,INSANALLY,NaN,﻿There is now abroad in the world a mood of op...
6579,66,2011,CUB,Cuba,Bruno Rodriguez Parrilla,Minister for Foreign Affairs,I am pleased to welcome the Government of \nSo...


time: 3.99 s (started: 2023-04-06 21:43:44 +00:00)


### 🚧 todo:
- use the CountVectorizer(ngram_range=(2, 3), min_df=10) to build a list of potential terms
    - raise min_df or lower max_features if you have an out of memory error
        - this is no exercice in auto-flagellation!
    - sort by frequency of occurrence of the n-gram
    - give the first rank where the n-gram is not a term
- use the "stop_words='english'" parameter
    - give the first rank where the n-gram is not a term
- explain why neither method is satisfactory

In [5]:
# Use CounVectorizer to build a list of potential terms
cvectorizer = CountVectorizer(ngram_range=(2, 3), min_df=10)
X=cvectorizer.fit_transform(df.text)
X

<7507x389472 sparse matrix of type '<class 'numpy.int64'>'
	with 22737299 stored elements in Compressed Sparse Row format>

time: 1min 27s (started: 2023-04-06 21:43:49 +00:00)


In [6]:
# (number of texts, number of n-grams)
X.shape

(7507, 389472)

time: 2.62 ms (started: 2023-04-06 21:45:17 +00:00)


In [7]:
# (1, number of n-grams)
np.sum(X, axis=0).shape

(1, 389472)

time: 118 ms (started: 2023-04-06 21:45:17 +00:00)


In [8]:
# Frequency of occurrence of every n-gram
Xdf = pd.DataFrame(np.sum(X, axis=0), 
                   columns=cvectorizer.get_feature_names_out()).T

# Sort by frequency of occurrence of the n-gram, from high to low
Xdf = Xdf.sort_values(by = 0, ascending = False)

Xdf.head(15)

,0
of the,397342
in the,154303
to the,118439
united nations,103514
the united,103063
the united nations,89150
and the,86232
for the,66418
on the,64648
that the,55059


time: 864 ms (started: 2023-04-06 21:45:17 +00:00)


In [9]:
# Reset index
Xdf = Xdf.reset_index().rename(columns={'index': 'n-gram'})
Xdf.head(15)

,n-gram,0
0,of the,397342
1,in the,154303
2,to the,118439
3,united nations,103514
4,the united,103063
5,the united nations,89150
6,and the,86232
7,for the,66418
8,on the,64648
9,that the,55059


time: 42.4 ms (started: 2023-04-06 21:45:18 +00:00)


<font color='orange'>
The first rank where the n-gram is not a term is just the first one: "of the".
<font>

In [10]:
# Add stopwords in CountVectorizer
cvectorizer_stopwords = CountVectorizer(ngram_range=(2, 3), min_df=10, stop_words='english')
X_stopwords = cvectorizer_stopwords.fit_transform(df.text)

# Sort by frequency of occurrence of the n-gram
X_stopwords_df = pd.DataFrame(np.sum(X_stopwords, axis=0), 
                   columns=cvectorizer_stopwords.get_feature_names_out()).T.sort_values(by=0, ascending=False)

# Reset index
X_stopwords_df = X_stopwords_df.reset_index().rename(columns={'index': 'n-gram'})
X_stopwords_df.head(15)

,n-gram,0
0,united nations,103521
1,international community,27779
2,general assembly,27112
3,security council,20986
4,human rights,20020
5,developing countries,18544
6,secretary general,17418
7,peace security,15244
8,united states,12569
9,middle east,12142


time: 1min 1s (started: 2023-04-06 21:45:18 +00:00)


<font color='orange'>
The first rank where the n-gram is not a term is the 8th one: "peace security".
<font>

Q: Why neither method(to build a list of potential terms) is satisfactory?

<font color='orange'>
A: 1) The extracted n-grams may not be meaningful terms as they may just be random combinations in the text. 
    
2) Some important terms may be missing from the vocabulary because they do not satisfy the threshold condition for n-grams, or because they are long phrases consisting of multiple words rather than simple n-grams.
<font>

### 🚧 todo:
#### quickly redo the same thing for the QA domain:

In [11]:
docs = pd.read_csv(path+'/our.text.msmarco.docs.tsv',sep='\t',usecols=[1,2]) 
docs[:999]

,docid,text
0,D2981241,What do you call a group of lions? Lions Vocab...
1,D687756,". The A Priori Argument ( also, Rationalizatio..."
2,D913099,Everything You Need To Learn How To Cook Veget...
3,D328017,"What is the difference between latitude, longi..."
4,D1636347,When was the pulley invented? Answers.com ® Wi...
...,...,...
994,D3342489,Travis Writes from the Alamo: âVictory or Deat...
995,D1789587,"MGM Resorts International From Wikipedia, the ..."
996,D98171,What are five properties of acids and bases? A...
997,D1744728,Welcome to the next generation of mineral wate...


time: 35.1 s (started: 2023-04-06 21:46:19 +00:00)


In [12]:
# Add stopwords in CountVectorizer
cvectorizer_stopwords = CountVectorizer(ngram_range=(2, 2), min_df=15, stop_words='english')
X_stopwords = cvectorizer_stopwords.fit_transform(docs.text)

# Sort by frequency of occurrence of the n-gram
X_stopwords_df = pd.DataFrame(np.sum(X_stopwords, axis=0), 
                   columns=cvectorizer_stopwords.get_feature_names_out()).T.sort_values(by=0, ascending=False)

# Reset index
X_stopwords_df = X_stopwords_df.reset_index().rename(columns={'index': 'n-gram'})
X_stopwords_df.head(15)

,n-gram,0
0,united states,84020
1,new york,61695
2,read dr,35957
3,et al,35485
4,archived original,32592
5,doi 10,30689
6,years ago,29094
7,http www,25709
8,long term,22322
9,blood pressure,21644


time: 6min 39s (started: 2023-04-06 21:46:55 +00:00)


<font color='orange'>
The first rank where the n-gram is not a term is the 3rd one: "read dr".
<font>

### 🚧 todo:
#### quickly redo the same thing for the patent domain:

In [13]:
# read patent file (already unzipped)
g06f=open(path+'/G06F.txt').read().strip()
patent_lines = g06f.split('\n')
print(len(patent_lines),'lines of patent text')

302426 lines of patent text
time: 2.55 s (started: 2023-04-06 21:53:34 +00:00)


In [14]:
X_stopwords=cvectorizer_stopwords.fit_transform(patent_lines)
# sort by frequency of occurrence of n-gram from high to low
X_stopwords_df = pd.DataFrame(np.sum(X_stopwords, axis=0), 
                   columns=cvectorizer_stopwords.get_feature_names_out()).T.sort_values(by=0, ascending=False)

# Reset index
X_stopwords_df = X_stopwords_df.reset_index().rename(columns={'index': 'n-gram'})
X_stopwords_df.head(15)

,n-gram,0
0,electronic device,28796
1,present invention,19388
2,present disclosure,17632
3,according embodiment,16176
4,user interface,14732
5,embodiment present,12729
6,shown fig,11232
7,mobile terminal,10840
8,control unit,10630
9,information processing,9704


time: 40.1 s (started: 2023-04-06 21:53:37 +00:00)


Q again: where do the first errors appear?

<font color='orange'>
A: The first rank where the n-gram is not a term is the 2nd one: "present invention".
<font>

### 🚧 todo:

Q: Compare this approach for the three domains (talks, QA, patents)

<font color='orange'>
A:  The effectiveness of the n-gram approach for term extraction and frequency sorting may vary depending on the domain of the text:  
    
Talks may benefit from the n-gram approach due to common phrases and colloquial language; QA texts may have technical terms and noise that reduce effectiveness; Patents may benefit from technical terms and standardized format, but have noise from legal jargon and references.
    
<font>

### 🤔 maybe a **term** is hard to define in statistical terms?

- we could do better: 
    - mutual information, 
    - specificity, 
    - entropy (at term borders)

- but let's first try to add information

## getting external lists

Wikipedia has
- page titles
- links (words pointing to pages)
- redirects (alternative page titles)

Wikipedia also has
- too many pages
- categories (not a taxonomy!)
- XML dumps

A ready-made list: 
- manyterms.lower.txt contains all these potential terms from these categories and subcategories (and some heuristics such as minimum frequency, some no-go Categories, ....): ['Category:Physics', 'Category:Technology', 'Category:Technology_by_type', 'Category:Techniques']
- 49166 categories, 2227891 pages
- 700k potential multi-word terms

Let's get a frequency count of these potential terms in our g06f!
That sounds easy!
- use 
    - a regex?
        - not working well 
    - a Tier
    - or CountVectorizer(vocabulary=...
    

In [15]:
# Here are the potential terms(mwe means 'multi-word expression')
mwes = open(path+'/manyterms.lower.txt').read().strip().split('\n') 
print(mwes[33333:33338])
print(len(mwes),'mwes')

['alpha-numeric code', 'alpha-particle emission', 'alpha-particle scattering experiment', 'alpha-particle spectroscopy', 'alpha-to-beta transition temperature']
743274 mwes
time: 649 ms (started: 2023-04-06 21:54:17 +00:00)


In [16]:
# not working: already crazy slow for 500 terms. Why?
n = 500
termlist = [re.escape(t) for t in mwes[:n]]
termregex = re.compile(r'\b'+r'\b|\b'.join(termlist), re.I)
termfreqs = Counter()
for i,li in tqdm(enumerate(g06f.split('\n'))):
    for m in termregex.finditer(li):
        termfreqs[m.group(0)]=termfreqs.get(m.group(0),0)+1
print(termfreqs.most_common(3))

0it [00:00, ?it/s]

[('10 seconds', 92), ('10 minutes', 84), ("10 o'clock", 7)]
time: 2min 42s (started: 2023-04-06 21:54:18 +00:00)


### 🚧 todo:
- explain in two sentences why previous approaches for this task are so slow and why turning to a Trie is more promising.  
<font color='orange'>
Regex approaches for this task are slow because they have to check every possible match in the text, which can be computationally expensive. 
Turning to a Trie is more promising because it can efficiently search for multiple patterns at once and avoid unnecessary comparisons.
<font>  
    
- understand the Trie algorithm
- put comments behind the lines that don't have any yet

In [17]:
# a Trie, working better:
class FreqMWETokenizer(MWETokenizer):
    """A tokenizer that processes tokenized text and merges multi-word expressions into single tokens.
    """
    
    def __init__(self, mwes=None, separator="_"):
        super().__init__(mwes, separator)

    def freqs(self, text):
        """
        :param text: A list containing tokenized text
        :type text: list(str)
        
        :return: A frequency dictionary with multi-words merged together as keys
        :rtype: dict
        
        :Example:
        >>> tokenizer = FreqMWETokenizer([ mw.split() for mw in ['multilayer ceramic', 'multilayer ceramic capacitor', 'ceramic capacitor']],
                                        separator=' ')
        >>> tokenizer.freqs("Gimme that multilayer ceramic capacitor please!".split())
        {'multilayer ceramic': 1, 'multilayer ceramic capacitor': 1, 'ceramic capacitor': 1}
        """
        i = 0  # index of token
        n = len(text) # number of tokens
        result = Counter()  # result stores the frequency of multi-word expressions(MWEs).

        while i < n:
            if text[i] in self._mwes:  # If the current token is in the list of MWEs.
                j = i
                trie = self._mwes  # Initialize the Trie object.
                
                while j < n and text[j] in trie: # Iterate over the tokens from i
                    if Trie.LEAF in trie: # If the current node is a leaf.
                        # Success! We found a MWE match.
                        mw = self._separator.join(text[i:j])  # Merge the MWE into a single token.
                        result[mw] = result.get(mw,0) + 1  # Increment the frequency of the MWE in the Counter.
                    
                    trie = trie[text[j]]  # Move one step down in the trie.
                    j += 1
                else:  # Executed if the while loop did not break.
                    if Trie.LEAF in trie:  # Check if the current node is a leaf.
                        # Success! We found a MWE match.
                        mw = self._separator.join(text[i:j])  # Merge this MWE into a single token.
                        result[mw] = result.get(mw,0) + 1  # Increment the frequency of this MWE in the Counter.
                    
                    i += 1
            else:
                i += 1

        return result


time: 3.76 ms (started: 2023-04-06 21:57:00 +00:00)


In [18]:
# Use example above to test FreqMWETokenizer 
tokenizer = FreqMWETokenizer([ mw.split() for mw in ['multilayer ceramic', 'multilayer ceramic capacitor', 'ceramic capacitor']],
                             separator=' ')
tokenizer.freqs("Gimme that multilayer ceramic capacitor please!".split())

Counter({'multilayer ceramic': 1,
         'multilayer ceramic capacitor': 1,
         'ceramic capacitor': 1})

time: 4.33 ms (started: 2023-04-06 21:57:00 +00:00)


In [19]:
# Apply Trie to data
splimwes = [mw.split() for mw in mwes]
tokenizer = FreqMWETokenizer(splimwes, separator=' ')
tokenizer.freqs("Gimme that multilayer ceramic capacitor please!".split())

Counter({'multilayer ceramic': 1,
         'multilayer ceramic capacitor': 1,
         'ceramic capacitor': 1})

time: 15 s (started: 2023-04-06 21:57:00 +00:00)


In [20]:
start = timeit.default_timer()
counter = tokenizer.freqs(g06f.split())
secs = timeit.default_timer()-start
print('it took', secs, 'seconds,', secs/len(mwes),'per term')


it took 37.23818162099997 seconds, 5.010020748875915e-05 per term
time: 37.2 s (started: 2023-04-06 21:57:15 +00:00)


In [21]:
g06fdf = pd.DataFrame.from_dict(counter, orient='index').sort_values(by = 0, ascending = False)
g06fdf

,0
the present,44578
electronic device,24272
number of,17680
the image,11835
user interface,11694
...,...
mammary carcinoma,1
phone lines,1
epidermal growth factor receptor 2,1
epidermal growth factor receptor,1


time: 18.6 ms (started: 2023-04-06 21:57:53 +00:00)


- is it clear why there are only 10k terms in the dataframe now?
<font color='orange'>
It remains only 15918 terms because only terms in the wikipedia list are counted as a term, not all n-grams.
<font>

- and now with the CountVectorizer

In [22]:
vectorizernb = CountVectorizer(vocabulary=mwes, binary=False, min_df=1, lowercase=True, ngram_range=(1,4))
Xnb = vectorizernb.fit_transform(patent_lines)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


time: 1min 28s (started: 2023-04-06 21:57:53 +00:00)


In [23]:
Xnb

<302426x743274 sparse matrix of type '<class 'numpy.int64'>'
	with 671191 stored elements in Compressed Sparse Row format>

time: 3.52 ms (started: 2023-04-06 21:59:22 +00:00)


In [24]:
Xnb.sum(axis=0).shape

(1, 743274)

time: 13.2 ms (started: 2023-04-06 21:59:22 +00:00)


In [25]:
vectorizernb.get_feature_names_out()[Xnb.sum(axis=0).argmax()]

'the present'

time: 533 ms (started: 2023-04-06 21:59:22 +00:00)


In [26]:
### 🚧 todo:
#     - find the terms from the lexicon with the 10 highest match in the patent texts

for i in np.array(np.argpartition(Xnb.sum(axis=0), -10))[0][-10:]:
    print(vectorizernb.get_feature_names_out()[i])

information processing
control unit
mobile terminal
the following
the system
the image
the present
electronic device
number of
user interface
time: 4.64 s (started: 2023-04-06 21:59:23 +00:00)


## preparing a Gold corpus

### 🚧 todo: 

- sort the patents of the G06F corpus into those that have reference numerals R and those that don't N
- count how many you have of each type
- for each patent of the R corpus, extract a list of numbered terms, verify manually for a few patents
- compare these terms with the ones detected using the previous methods
- later, in the spacy section: visualize the annotations using Spacy


In [27]:
# create an empty dataframe with predefined columns
patent_df = pd.DataFrame(columns=["title", "text", "type", "terms"])
patent_df

,title,text,type,terms


time: 15.3 ms (started: 2023-04-06 21:59:27 +00:00)


In [28]:
# Initialize variables
newpatent = True
title = ""
text = ""
terms = {}

# Loop over the patent_lines
i = 0
while i+1 < len(patent_lines):
    if newpatent:
        title = patent_lines[i]
        text = ""
        terms = {}
        i+=1
        newpatent = False
        
    elif patent_lines[i] == "" and patent_lines[i+1] == "":
        newpatent = True
        
        # Create a dictionary with most frequent term for each term id
        truetermdict = {}
        for termid in terms:
            freq = defaultdict(int)

            # Count frequency of each possible term
            for c in terms[termid]:
                for j in range(len(c)):
                    term = ' '.join(c[j:])
                    freq[term] += 1

            # If no terms were found, skip to the next one
            if not freq:
                continue
            
            # The most frequent term
            most_likely_term = max(freq, key=freq.get)
            
            # Filter out terms with "FIG" or "FIGS"
            if not ("FIG" in most_likely_term or "FIGS" in most_likely_term):
                truetermdict[termid] = most_likely_term
        
        # Add a new row to the dataframe with the processed data
        patent_df.loc[len(patent_df)] = [
            title,
            text,
            "R" if truetermdict else "N",
            truetermdict
        ]
        i += 1
        
    else:
        # Add current line to the text of the patent
        text += patent_lines[i]+"\n"
        
        # Process potential term ids
        words = patent_lines[i].split()
        for j in range(len(words)):
            if words[j].isnumeric():
                # Potential term id
                try:
                    termid = int(words[j])
                except ValueError:
                    continue
                
                # Create a list of words that precede the term id
                if not termid in terms:
                    terms[termid] = []
                toadd = words[max(0, j-4):j]
                if toadd not in terms[termid]:
                    terms[termid].append(words[max(0, j-4):j])
    i += 1

time: 15.7 s (started: 2023-04-06 21:59:27 +00:00)


In [29]:
# The terms dictionary
patent_df.terms[0]

{102: 'The model',
 116: 'coil',
 115: 'cooler',
 105: 'chiller',
 108: 'water',
 112: 'from the storage tank',
 119: 'water from the cooler',
 110: 'storage tank',
 118: 'bypass loop',
 106: 'to form the water',
 104: 'The model',
 138: 'Air',
 130: 'the plenum',
 140: 'temperature of Tp. Air',
 120: 'the room',
 142: 'air',
 144: 'Trm, to form air',
 135: 'IT equipment',
 146: 'Air',
 136: 'form mixed room air',
 124: 'walls',
 122: 'ceiling',
 126: 'the floor',
 103: 'combined model',
 10: 'is expressed in Equation',
 650: 'the input section',
 400: 'process',
 18: 'used to solve the',
 454: 'the chiller q˙chmax at',
 458: 'the chiller ṁch at',
 452: 'and begins again at',
 13: 'lists the coefficients for',
 100: 'where heat energy (e.g.,',
 20: 'data in the system',
 150: 'after the power outage.',
 600: 'screen',
 654: 'The cooling equipment input',
 656: 'equipment start time input',
 658: 'A simulation input subsection',
 660: 'The example results section',
 33: 'which is set a

time: 5.4 ms (started: 2023-04-06 21:59:43 +00:00)


In [30]:
# Count how many we have for each type
patent_df.type.value_counts()

R    1993
N       5
Name: type, dtype: int64

time: 9.41 ms (started: 2023-04-06 21:59:43 +00:00)


## 🪐 back to spacy
https://spacy.io/usage/rule-based-matching#phrasematcher

In [31]:
import spacy
from spacy.matcher import PhraseMatcher

# load vocab
nlp = spacy.load("en_core_web_sm")
# init matcher with the vocab
matcher = PhraseMatcher(nlp.vocab)
terms = ["Barack Obama", "Angela Merkel", "Washington, D.C."]
# [doc] If you only need the tokenization and lexical attributes, you can run nlp.make_doc
patterns = [nlp.make_doc(text) for text in terms]
# add patterns to the matcher
matcher.add("TerminologyList", patterns)

# init string to be matched
doc = nlp("German Chancellor Angela Merkel and US President Barack Obama "
          "converse in the Oval Office inside the White House in Washington, D.C.")

# get matches
matches = matcher(doc)

# iterate over the matches and print the span text
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

Angela Merkel
Barack Obama
Washington, D.C.
time: 999 ms (started: 2023-04-06 21:59:43 +00:00)


In [32]:
from spacy import displacy
displacy.render(doc, style="ent")

time: 4.04 ms (started: 2023-04-06 21:59:44 +00:00)


In SpaCy, the pretrained default pipeline may consist of various componenets. If we want to train specific pipeline, it can be initiated as a blank and we can add only necessary elements.

In [33]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x76331917b6e0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x76331917b750>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x76332f562250>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x763448016f00>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7633cf420b40>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7633c0d17f50>)]

time: 2.87 ms (started: 2023-04-06 21:59:45 +00:00)


In [34]:
# !python -m spacy download en_core_web_lg
g06f=open(path+'/G06F.txt').read().strip()

from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span

patent_texts = g06f.split('\n\n')
patent_lines = g06f.split('\n')

nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000]) # 
displacy.render(doc, style="ent", jupyter = True)

time: 6.19 s (started: 2023-04-06 21:59:45 +00:00)


In [35]:
# if restarting here: (17 secs)
from spacy.matcher import PhraseMatcher

g06f_lines = g06f.split('\n')
print(len(g06f_lines),'lines of patent text')
cvectorizer = CountVectorizer(ngram_range=(2, 3), stop_words='english', min_df=500,) # , stop_words='english'  max_features
X=cvectorizer.fit_transform(g06f_lines)
Xdf = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names_out()).T.sort_values(by = 0, ascending = False)


302426 lines of patent text
time: 1min 1s (started: 2023-04-06 21:59:51 +00:00)


### 🚧 todo: explain in one line what each of the following cells does (up to the next existing comment)

<font color='orange'>
This cell initializes a PhraseMatcher with the spaCy pipeline vocabulary and adds patterns as documents to match against with the label "Tech".
<font>

In [36]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in Xdf.index]
matcher.add("Tech", patterns)

time: 45.3 ms (started: 2023-04-06 22:00:53 +00:00)


<font color='orange'>
This cell splits the patent_lines list into two sublists: train_lines and test_lines with a test size of 30%.
<font>

In [37]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(patent_lines, test_size=0.3, random_state=42)


time: 132 ms (started: 2023-04-06 22:00:53 +00:00)


<font color='orange'>
This cell iterates over the first 30,000 lines of train_lines, converts each line into a spaCy Doc object, and adds entities (matching with the PhraseMatcher) to each document, which are then filtered and added to doc_bin_train.
<font>

In [38]:
LABEL = "TT"
doc_bin_train = DocBin() # create a DocBin object

# nlp.max_length = 2000000
for training_example  in tqdm(train_lines[:30000]): #~50 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    #print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_train.add(doc)

  0%|          | 0/30000 [00:00<?, ?it/s]

time: 22.8 s (started: 2023-04-06 22:00:53 +00:00)


<font color='orange'>
This cell uses the trained PhraseMatcher to extract entities from the test_lines data and adds them to a DocBin object for validation, limiting the number of patents processed to 15.
<font>

In [39]:
LABEL = "TT"
doc_bin_valid = DocBin() # create a DocBin object
nlp = spacy.blank("en")
# nlp.max_length = 2000000
for training_example  in tqdm(test_lines[:10000]): #~15 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    #print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_valid.add(doc)

  0%|          | 0/10000 [00:00<?, ?it/s]

time: 8.14 s (started: 2023-04-06 22:01:16 +00:00)


<font color='orange'>
This cell iterates over a portion of the test_lines list, filters the entities and adds the filtered Doc object to the doc_bin_test DocBin.
<font>

In [40]:
LABEL = "TT"
doc_bin_test = DocBin() # create a DocBin object
nlp = spacy.blank("en")
# nlp.max_length = 2000000
for training_example  in tqdm(test_lines[10000:12000]): #~5 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    #print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_test.add(doc)

  0%|          | 0/2000 [00:00<?, ?it/s]

time: 2.07 s (started: 2023-04-06 22:01:25 +00:00)


<font color='orange'>
This cell saves the DocBin objects containing the training, validation, and testing data in spaCy format, and saves the training, validation, and testing data as text files.
<font>

In [41]:
doc_bin_train.to_disk("training_data.spacy") # save the docbin object
doc_bin_valid.to_disk("valid_data.spacy") # save the docbin object
doc_bin_test.to_disk("test_data.spacy") # save the docbin object

# save train_lines to txt file
with open('train_lines.txt', 'w') as f:
    for line in train_lines:
        f.write(line)
        f.write('\n')
f.close()

# save train_lines to txt file
with open('valid_lines.txt', 'w') as f:
    for line in test_lines[:10000]:
        f.write(line)
        f.write('\n')
f.close()

# save test_lines to txt file
with open('test_lines.txt', 'w') as f:
    for line in test_lines[10000:12000]:
        f.write(line)
        f.write('\n')
f.close()

time: 8.29 s (started: 2023-04-06 22:01:27 +00:00)


Donwnload __base_config.cfg__ for your system at https://spacy.io/usage/training#quickstart

In [42]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1znItIeR-868aeNihbAMSlwwvFbnQyamk' -O base_config.cfg

--2023-04-06 22:01:37--  https://docs.google.com/uc?export=download&id=1znItIeR-868aeNihbAMSlwwvFbnQyamk
Resolving docs.google.com (docs.google.com)... 74.125.197.101, 74.125.197.113, 74.125.197.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.197.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/17pl3pfv9mgup611tpjvcptk4dsl8t0g/1680818475000/12415620844995071159/*/1znItIeR-868aeNihbAMSlwwvFbnQyamk?e=download&uuid=774e046d-37c0-41bb-b2ce-7a89c42bf97f [following]
--2023-04-06 22:01:37--  https://doc-0c-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/17pl3pfv9mgup611tpjvcptk4dsl8t0g/1680818475000/12415620844995071159/*/1znItIeR-868aeNihbAMSlwwvFbnQyamk?e=download&uuid=774e046d-37c0-41bb-b2ce-7a89c42bf97f
Resolving doc-0c-9o-docs.googleusercontent.com (doc-0c-9o-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:

In [43]:
# Run to generate full training config
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
time: 17 s (started: 2023-04-06 22:01:37 +00:00)


#### Training
Run training. All results are stored into __./spacy_output__ 

In [44]:
# takes some time!
# if it crashes VS Code you may want to try to run this line directly in the command line
!python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy

✔ Created output directory: spacy_output
ℹ Saving to output directory: spacy_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-06 22:02:04,534] [INFO] Set up nlp object from config
[2023-04-06 22:02:04,550] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-06 22:02:04,557] [INFO] Created vocabulary
[2023-04-06 22:02:07,090] [INFO] Added vectors: en_core_web_lg
[2023-04-06 22:02:11,016] [INFO] Finished initializing nlp object
[2023-04-06 22:02:57,020] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    137.17    0.87    0.55    2.12    0.01
  0     200         13.56   1647.98   41.35   32.19   57.80    0.41
 

### Testing

In [45]:
# Load best model
nlp_ner = spacy.load("./spacy_output/model-best")

# Just text snippet
doc = nlp_ner("Web scraping or web data extraction methods are known in the art. Web scraping is used to access the World Wide Web directly using the Hypertext Transfer Protocol, or through a web browser. Web scraping typically refers to automated processes implemented using a bot or web crawler. It is a form of copying, in which specific data is gathered and copied from the web, typically into a central local database or spreadsheet, for later retrieval or analysis.\
Web scraping a web page involves retrieving a predefined Hypertext Markup Language (HTML) page and extracting data from it. Fetching is the downloading of a page which is stored under a static Web address typically specified by a Uniform Resource Locator (URL). Once the page is fetched from where it had been stored, extraction can take place. The content of a page may then be parsed, searched, reformatted, etc. Web scrapers typically extract certain parts of a page to make use \
of it for another purpose. An example is to find and copy names and phone numbers, \
or companies and their URLs, to a list (so-called contact scraping).\
Prior art Web scraping tools can retrieve web page content from pages which are stored as predefined HTML data. Such content is referred to as static content herein because it relates to content provided by static web pages. However, current web technology allows to dynamically generate web pages on a web server in response to requests which may be received from a user or a computer system. As a consequence, data shown on websites can continuously change. A web page containing respective data can change \
its layout and new data fields may be introduced at any time. The content of such dynamic web pages (dynamic content) typically depends on the navigation history through a website. In other words, it depends on where the user currently is and which information and requests have been sent previously.\
 Current Web scraping tools fail to scrape dynamic content data from such dynamically generated web pages and provide respective content data in a machine-readable format so that the content can be further processed by other computer systems provided with the extracted data.\
Summary\
Hence, there is a need for providing improved methods and systems to enable web scraping for dynamic content on dynamic web pages. \
This technical problem is solved by a computer system, a computer-implemented method and a computer program product as disclosed in the independent claims. The disclosed embodiments define a screen-scraping framework which addresses the above problem by automatically connecting to a target website and extracting dynamic data from said target website.\
")

# Show NER results
spacy.displacy.render(doc, style="ent", jupyter=True)

time: 7.64 s (started: 2023-04-07 06:36:34 +00:00)
